In [2]:
import numpy as np
import random
from random import randint
from numpy.linalg import norm
from random import gauss
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import time

#from sklearn.manifold import TSNE

In [3]:
def getvector(givennorm, d) :   
    vec = [gauss(0, 1) for i in range(d)]
    mag = sum(x**2 for x in vec) ** .5
    return [(givennorm*x)/mag for x in vec]

In [4]:
def createinputmatrix(n, d, givennorm):
    v = np.empty([0, 100000])
    for i in range(n):
        norm1 = randint(a = 1, b = givennorm)
        vn = getvector(norm1, d)
        v = np.append(arr = v, values = [vn], axis = 0)
    return v
    

In [5]:
def cointoss(n):
    cointossvector = []
    for i in range(n):
        toss = randint(1, 2)
        if(toss == 1):
            cointossvector.append(1)
        elif(toss == 2):
            cointossvector.append(-1)
    return cointossvector

In [6]:
def getJLmatrix( reduceddim, actualdimensions):
    jlmatrix = np.empty([0, actualdimensions])
    for i in range(reduceddim):
        jlmatrix = np.append(arr = jlmatrix, values = [cointoss(actualdimensions)], axis = 0)
    return jlmatrix

In [7]:
def jltransform(reduceddim, matrix):
    start = time.time()
    jlmatrix = getJLmatrix(reduceddim, matrix.shape[1])
    jlmatrix = np.transpose(jlmatrix)
    compressedmatrix = np.dot(matrix, jlmatrix)
    end = time.time()
    print(end - start)
    return compressedmatrix
        
    

In [8]:
def clusterintersections(original, matrix, numclusters):
    start = time.time()
    km = KMeans(n_clusters = numclusters)
    originalclustering = km.fit(original)
    km = KMeans(n_clusters = numclusters)
    jlclustering = km.fit(matrix)
    oc = np.zeros([numclusters, matrix.shape[0]])
    jlc = np.zeros([numclusters, matrix.shape[0]])
    ip = []
    for i in range(matrix.shape[0]):
        oc[originalclustering.labels_[i],i] = 1
        jlc[jlclustering.labels_[i],i] = 1
    mxdp = []    
    for i in range(numclusters):
        dp = []
        for j in range(numclusters):
            dp.append(np.inner(oc[j],jlc[i]))
        mxdp.append(max(dp))
    end = time.time()
    print(end - start)
    return mxdp

In [9]:
def accuracy(result, matrix):
    return((sum(result)/matrix.shape[0])*100)

In [10]:
def ourtransform(reduceddim, matrix):
    start = time.time()
    mapping = []
    compressedmatrix = np.empty([0,reduceddim])
    for i in range(matrix.shape[1]):
        toss = randint(0, reduceddim-1)
        mapping.append(toss)
    for i in range(matrix.shape[0]):
        reducedvector = []
        vectoraftermapping = np.zeros([reduceddim,matrix.shape[1]])
        vector = matrix[i]
        randomvector = cointoss(matrix.shape[1])
        vector = np.multiply(vector,randomvector)
        #print("rv",vector.shape)
        for j in range(matrix.shape[1]):
            vectoraftermapping[mapping[j],j] = vector[j]
            #print(j)
        for k in range(reduceddim):
            reducedvector.append(sum(vectoraftermapping[k]))
        reducedvector = np.array(reducedvector)
        compressedmatrix = np.append(arr = compressedmatrix, axis = 0, values = [reducedvector])
    end = time.time()
    print(end - start)
    return compressedmatrix
        

In [11]:
inputmatrix = createinputmatrix(500, 100000, 1000)

In [ ]:
ourresult5 = ourtransform(5, inputmatrix)

In [ ]:
ourresult1 = clusterintersections(inputmatrix, ourresult, 5)

In [ ]:
ourresult1

In [ ]:
print(accuracy(ourresult1, inputmatrix))

In [ ]:
jlresult = jltransform(100, inputmatrix)

In [ ]:
jlresult1 = clusterintersections(inputmatrix, jlresult, 5)

In [ ]:
jlresult1

In [ ]:
print(accuracy(jlresult1, inputmatrix))